In [1]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

### Q1. Downloading the data

In [2]:
!ls -ls

total 93224
    4 -rw-rw-rw- 1 codespace root          2100 May 20 18:22 homework.md
   20 -rw-rw-rw- 1 codespace codespace    20115 May 23 15:43 homework1.ipynb
46564 -rw-rw-rw- 1 codespace codespace 47673370 Mar 20  2023 yellow_tripdata_2023-01.parquet
46636 -rw-rw-rw- 1 codespace codespace 47748012 May  1  2023 yellow_tripdata_2023-02.parquet


In [4]:
!rm *.parquet.*
!rm *.parquet

rm: cannot remove '*.parquet.*': No such file or directory
rm: cannot remove '*.parquet': No such file or directory


In [5]:
!ls -ls

total 24
 4 -rw-rw-rw- 1 codespace root       2100 May 20 18:22 homework.md
20 -rw-rw-rw- 1 codespace codespace 20115 May 23 15:43 homework1.ipynb


In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet 

--2024-05-23 15:44:33--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.119, 18.239.238.212, 18.239.238.133, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M   216MB/s    in 0.2s    

2024-05-23 15:44:33 (216 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]

--2024-05-23 15:44:34--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.239.238.212, 18.239.238.133, 18.239.238.152, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.239.238.212|:443... 

In [7]:
!ls -ls

total 93216
    4 -rw-rw-rw- 1 codespace root          2100 May 20 18:22 homework.md
   20 -rw-rw-rw- 1 codespace codespace    20115 May 23 15:43 homework1.ipynb
46560 -rw-rw-rw- 1 codespace codespace 47673370 Mar 20  2023 yellow_tripdata_2023-01.parquet
46632 -rw-rw-rw- 1 codespace codespace 47748012 May  1  2023 yellow_tripdata_2023-02.parquet


In [8]:
!pwd

/workspaces/mlop/01-intro


In [9]:
df1 = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df2 = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [10]:
print(f"Q1-January has {len(df1.columns)} Columns.")

Q1-January has 19 Columns.


## Q2. Computing duration

In [11]:
df1['duration'] = df1.tpep_dropoff_datetime - df1.tpep_pickup_datetime
df1['duration'] = df1.duration.dt.total_seconds() / 60

In [12]:
print(f"Q2-The Standard Deviation of the trips duration in January 2023 {df1.duration.std()}")

Q2-The Standard Deviation of the trips duration in January 2023 42.594351241920904


## Q3. Dropping outliers
* Check the distribution of the Duration variable.
* Remove some outliers.
* Keep records between 1 and 60 minutes.
* What fraction of the records left after dropping the outliers?

In [13]:
fractPer = len(df1[(df1.duration >= 1) & (df1.duration <= 60)]) / len(df1) * 100

In [14]:
print(f"Q3-The Fraction of records left after the January 2023: {fractPer}")

Q3-The Fraction of records left after the January 2023: 98.1220282212598


In [15]:
df = df1[(df1.duration >= 1) & (df1.duration <= 60)].copy()

## Q4. One-hot encoding
* Apply one-hot encoding to the pickup and dropoff location IDs.
* Turn the dataframe into a list of dictionaries 
* Fit a dictionary vectorizer 
* Get a feature matrix from it
* What's the dimensionality of this matrix (number of columns)?


In [16]:
categorical = ['PULocationID', 'DOLocationID']

In [17]:
df[categorical] = df[categorical].astype(str)

In [18]:
train_dicts = df[categorical].to_dict(orient='records')

In [32]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [21]:
print(f'Feature matrix size: {X_train.shape}')

Feature matrix size: (3009173, 515)


## Q5. Training a model
* Now let's use the feature matrix from the previous step to train a model. 
* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

In [22]:
target = 'duration'
y_train = df[target].values

In [23]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print(f'Train RMSE: {mean_squared_error(y_train, y_pred, squared=False)}')

Train RMSE: 7.649261027782827


## Q6. Evaluating the model
* Now let's apply this model to the validation dataset (February 2023).
* What's the RMSE on validation?

In [24]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].astype('str')
    
    return df

In [30]:
!ls -ls

total 93224
    4 -rw-rw-rw- 1 codespace root          2100 May 20 18:22 homework.md
   20 -rw-rw-rw- 1 codespace codespace    17205 May 23 15:55 homework1.ipynb
46564 -rw-rw-rw- 1 codespace codespace 47673370 Mar 20  2023 yellow_tripdata_2023-01.parquet
46636 -rw-rw-rw- 1 codespace codespace 47748012 May  1  2023 yellow_tripdata_2023-02.parquet


In [31]:
df_val = read_data('/workspaces/mlop/01-intro/yellow_tripdata_2023-02.parquet')

In [33]:
df_val.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee', 'duration'],
      dtype='object')

In [34]:
val_dicts = df_val[categorical].to_dict(orient='records')

In [35]:
X_val = dv.transform(val_dicts) 
y_val = df_val.duration.values

In [36]:
X_val.shape

(2855951, 515)

In [37]:
y_pred = lr.predict(X_val)

In [38]:
print(f'Val RMSE: {mean_squared_error(y_val, y_pred, squared=False)}')

Val RMSE: 7.8118326159414675
